
# Workshop about controllers
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents
* Writing controller - general
* Writing motor controller - Blender blades
* Writing pseudo motor controller - Slits
* Writing counter/timer controller - Network Trafic Counter

### Before writing a new controller
* Before writing a new controller check in the [third-party repository](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/) if someone already wrote a similar one e.g.
 * Motor: IcePAP, Pmac\*, PIE516, Linkam TST350\*, Newport NSC200, Oxford Cryostream 700\*, Smaract MCS, OmsVme58\*, NewFocus\*, Kohzu\*, AttoCube\*, Galil\*, Micos\*, Aerotech\*, OwisMMS19Tango\*, Tango attribute, ...
 * PseudoMotor: Tripod, Tables, Translatations, Energy, ...
 * IORegister: SIS3610\*, Pmac\*, ...
 * CounterTimer: Adlink2005\* , NI6602\*, AlbaEm\*, DGG2\*, SIS3820\*, ROIs, Tango attribute, ...
 * 1D: AmptekPX5\*, Mythen\*, SIS3302, ...
 * 2D: PCO\*, Eiger\*, Lambda\*, MarCCD\*, Pilatus\*, PerkinElmer\*, Lima\*, ...
* If not, it does not harm to ask on one of the communication channels e.g. mailing list, github issue, etc.

\* Communicate via Tango devices

### Controller plugins discovery
* Sardana built-in plugis are importable from `sardana.pool.poolcontrollers`
* The plugin discovery system is based on directory scanning and python module inspection
* Custom controllers should be installed in one of the `PoolPath` directories:
 * Create /controllers directory: `mkdir /controllers`
 * In spock: `Pool_demo1_1.put_property({"PoolPath":["/controllers"]})`
 * Restart the Sardana server
* The path order is important! Controllers in the higher position paths will take precedence over the lower position paths.

### Blender Blades Motor Controller

* Open the code: `kwrite sardana-training/controllers/templates/BlenderBladesMotorCtrl.py`

### Blender Blades Motor Controller

* Requisites: blender (in Debian: `apt-get install blender`)
* Start hardware: `blenderplayer sardana-training/blender_slits/slits.blend`
* Blender Blades system:
 * Communication protocal is explained in: `sardana-training/blender_slits/comm_protocol.txt`
 * Starts at completelly closed slits
 * Directions are in the hardware coordinate system
 * Positions are not callibrated in the local coordinate system - beam axis is the zero

### [Writing constructor](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_controller.html#constructor)
* Constructor
 * Called on: controllers creation, pool startup and controller's code reload
 * Accepts arguments: instance (name of the controller instance) and properties (dictionary with the controller properties)
 * If an exception is raised when constructing the controller, the controller automatically gets into the Fault state and its status describes the exception that occured
* Instantiate `BlenderBlades` in the constructor

### Instantiate controller
* Deploy controller: `ln -s /sardana-training/controllers/templates/BlenderBladesMotorCtrl.py /controllers`
* Load it in the system - in spock: `addctrllib BlenderBladesMotorCtrl`
* Create an instance of the controller: `defctrl BlenderBladesMotorController bleblactrl`
* Ask for controllers state: `bleblactrl.state()`

### [Implement StateOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html#get-motor-state)
* Get axis state (State sequence)
 * Applies only to the to physical elements
 * Called on: state request, during operations e.g. motion, acquisition
 * Returns: state and optionally status, if no status is returned, it will be composed by Sardana from the state (in case of motor also returns limit switches)
 * If an exception is raised when reading the state, the axis automatically gets into the Fault state and the status contains the exception details.

### [Implement StateOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html#get-motor-state)
* Implement AXIS_NAMES class member for quick lookup to motor identifiers
* Implement AXIS_ID and VALUE class members for refering to the corresponing part of the answer
* Implement method by quering `?state <axis id>`

### [Implement ReadOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html#get-motor-position)
* Implement method by quering `?pos <axis id>`

### Instantiate top motor
* Reload controller code: `relctrlcls BlenderBladesMotorController` 
* Create motor instance: `defelem top bleblactrl 1`
* Ask for motor state: `top.state()`
* Ask for motor position: `top.position` or `wm top`

### [Implement StartOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html#move-a-motor)
* Implement method by commanding `<axis id> <pos>`

### Move top motor
* Reload controller code: `relctrlcls BlenderBladesMotorController` 
* Ask motor to move: `umvr top 10`

### [Implement AbortOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html#abort-a-motor)
* Implement method by commanding `abort`

### Move and abort top motor
* Reload controller code: `relctrlcls BlenderBladesMotorController` 
* Ask motor to move: `umvr top 50`
* Issue `Ctrl+C` in spock

### [Implement standard axis attributes](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html#standard-axis-attributes) - Homework:)
* Implement `GetAxisPar` method
 * Implement velocity by querying `?vel <axis id>`
 * Implement acceleration by querying `?acc <axis id>`
 * Implement deceleration by querying `?dec <axis id>`
 * Hardcode base_rate to 0
 * Hardcode steps_per_unit to 1

### Read standard axis attributes
* `top.velocity`
* `top.acceleration`
* `top.deceleration`
* `top.base_rate`
* `top.state_per_unit`

### [Implement standard axis attributes](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html#standard-axis-attributes) - Homework:)
* Implement `setAxisPar` method
 * Implement velocity by commanding `vel <axis id> <value>`
 * Implement acceleration by commanding `acc <axis id> <value>`
 * Implement deceleration by commanding `dec <axis id> <value>`
 * Raise exception when base_rate is set
 * Raise exception when steps_per_unit is set
 
 

### Write standard axis attributes
* `top.velocity = 10`
* `top.acceleration = 0.1`
* `top.deceleration = 0.1`
* `top.base_rate = 5` -> exception!
* `top.state_per_unit = 100` -> -> exception!
* Make some moves...

### Align slits
* Define rest of the motors:
 * `defm bottom bleblactrl 2`
 * `defm left bleblactrl 3`
 * `defm right bleblactrl 4`
* Ask for motor positions: `wm top bottom left right`
* Determine direction with relative move and adjust sign:
 * `bottom.sign = -1`
 * `left.sign = -1`
* Change offset by using the `set_user_pos` macro e.g. `set_user_pos top 0`
* Open and close gap by moving physical motors
* Move offset by moving the physical motors

### Slits pseudo motor controller

* Open the code: `kwrite sardana-training/controllers/BlenderSlitCtrl.py`

### Slits pseudo motor controller - Homework:)
* `pseudo_motor_roles` and `motor_roles` class members defines number of pseudo and physical axes used by the controller
* Implement `CalcPhysical` method:
 * Calculate half_gap
 * top = offset + half_gap 
 * Bottom would be offset - half_gap if directions were common
 * Since the directions are opposit bottom = half_gap - offset
* Implement `CalcPseudo` method:
 * gap = bottom + top
 * offset = top - half_gap

### Slits pseudo motor controller
* Deploy controller: `ln -s /sardana-training/controllers/BlenderSlitCtrl.py /controllers`
* Load it in the system, in spock: `addctrllib BlenderSlitCtrl`
* Instantiate vertical slits controller: `defctrl BlenderSlit vertctrl top=top bottom=bottom gap=gapvert offset=offsetvert`
* Instantiate horizontal slits controller: `defctrl BlenderSlit horctrl top=right bottom=left gap=gaphor offset=offsethor`

### Network Traffic Counter

* Open the code: `kwrite sardana-training/controllers/templates/NetworkTrafficCTCtrl.py`

### Network Traffic Counter
* Network traffic counter counts bytes that pass via a network interface (both ways)
* Demo:
 * Directly from the system: `cat /proc/net/dev | grep eth0`
 * Using python function:
   * `cd /sardana-training/controllers/`
   * `python -c "from NetworkTrafficCTCtrl import read_network_counts; print read_network_counts('eth0')"`

### [Writing constructor](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_controller.html#constructor)
* Constructor
 * Called on: controllers creation, pool startup and controller's code reload
 * Accepts arguments: instance (name of the controller instance) and properties (dictionary with the controller properties)
 * If an exception is raised when constructing the controller, the controller automatically gets into the Fault state and its status describes the exception that occured
* [Define controller property](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_controller.html#extra-controller-properties) `interface` - Homework:)

### [Writing constructor](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_controller.html#constructor)
* Counter will work on the following principle:
 * Load of the timer will store the integration time in the cache
 * Start of the counting will latch the current number of bytes
 * Start of the counting will define the end acquisition time (current time + integration time)
* In the constructor we need to instantiate the necessary variables:
 * acq_time = 1
 * acq_end_time = time.time()
 * start_counts = 0

### Instantiate controller
* Deploy controller: `ln -s /sardana-training/controllers/templates/NetworkTrafficCTCtrl.py /controllers`
* Load it in the system - in spock: `addctrllib NetworkTrafficCTCtrl`
* Create an instance of the controller: `defctrl NetworkTrafficCounterTimerController netctrl interface eth0`
* Ask for controllers state: `netctrl.state()`

### [Implement StateOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_controller.html#get-axis-state)
* Assume state is On
* Only if current time is less then the acq_end_time return Moving state
* IMPORTANT: due to [sardana-org/sardana#621](https://github.com/sardana-org/sardana/issues/621) return an empty status `""`

### [Implement LoadOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_countertimercontroller.html#load-a-counter)
 * receives either integration time or monitor counts (negative number) as arguments \*
 * is called only on the controller's timer or monitor
 
 * store the integration time in the cache `acq_time`
 
\* An advanced API is available for continuous acquisitions e.g. continuous scans

### [Implement ReadOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_countertimercontroller.html#get-counter-value)
 * returns a single counter value
 * is called multiple times during the acquisition operation

 * latch network bytes
 * return a difference between the current network bytes and `start_counts`
 
\* An advanced API is available for continuous acquisitions e.g. continuous scans

### Instantiate net counter
* Reload controller code: `relctrlcls NetworkTrafficCounterTimerController` 
* Create motor instance: `defelem net netctrl 1`
* Ask for motor state: `net.state()`
* Ask for counter value: `net.value`

### [Implement StartOne](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_countertimercontroller.html#start-a-counter)
 * receives intergation timer or monitor count as argument
 * latch the current bytes to `start_counts` and calcultate the end acquisition time in `acq_end_time`
 * IMPORTANT: Due to [sardana-org/sardana#622](https://github.com/sardana-org/sardana/issues/622) `StartAll` is mandatory


### Demonstrate single acquisition with net counter
* Reload controller code: `relctrlcls NetworkTrafficCounterTimerController` 
* Create measurement group: `defmeas mntgrp04`
* Set the newly created measurement group as active: `senv ActiveMntGrp mntgrp04`
* ct -> zero counts!
* Install `wget` tool and get sardana CHANGELOG while long counting e.g. `ct 10` execute:
 * `wget https://github.com/sardana-org/sardana/blob/develop/CHANGELOG.md`